In [5]:
from typing import Self

In [76]:
class Block:
    def __init__(self, start: tuple[int, int, int], end: tuple[int, int, int], tag: str|None = None) -> None:
        if start[2] <= end[2]:
            self.start = start
            self.end = end
        else:
            self.start = end
            self.end = start
        self.tag = tag

    def __lt__(self, other: Self) -> bool:
        return self.start[2] < other.start[2]
    
    def __repr__(self) -> str:
        if self.tag is not None:
            return f"{self.tag}\t{self.start[0]},{self.start[1]},{self.start[2]}~{self.end[0]},{self.end[1]},{self.end[2]}"
        else:
            return f"{self.start[0]},{self.start[1]},{self.start[2]}~{self.end[0]},{self.end[1]},{self.end[2]}"
    
    def drop1(self) -> Self:
        return Block((self.start[0], self.start[1], self.start[2]-1), (self.end[0], self.end[1], self.end[2]-1), self.tag)
    
    def coords(self) -> set[tuple[int, int, int]]:
        return {(x, y, z) for x in range(self.start[0], self.end[0]+1) for y in range(self.start[1], self.end[1]+1) for z in range(self.start[2], self.end[2]+1)}

    @classmethod
    def parse(cls, text: str, tag: str|None = None) -> Self:
        start, end = text.split('~')
        return Block(tuple(int(i) for i in start.split(',')), tuple(int(i) for i in end.split(',')), tag)

In [143]:
def dropblocks(blocks: list[Block]) -> tuple[list[Block], int]:
    it = 0
    sblocks = sorted(blocks)
    settled: list[Block] = []
    maxheight: dict[tuple[int, int], int] = {}
    moves = 0
    while len(sblocks) > 0:
        it += 1
        b = sblocks[0]
        if b.start[2] == 1:
            settled.append(b)
            sblocks = sblocks[1:]
            for c in b.coords():
                maxheight[(c[0], c[1])] = 1
            continue
        dropped = b.drop1()
        intersects = False
        for c in dropped.coords():
            if (c[0], c[1]) in maxheight.keys() and c[2] <= maxheight[(c[0], c[1])]:
                intersects = True
        if not intersects:
            sblocks = [dropped,] + sblocks[1:]
            moves += 1
        else:
            settled.append(b)
            sblocks = sblocks[1:]
            for c in b.coords():
                if (c[0], c[1]) in maxheight.keys():
                    maxheight[(c[0], c[1])] = max(c[2], maxheight[(c[0], c[1])])
                else:
                    maxheight[(c[0], c[1])] = c[2]
    return sorted(settled), moves


In [27]:
with open('test.txt', 'rt') as f:
    test = f.readlines()

In [124]:
blocks = [Block.parse(s.strip(), tag) for s, tag in zip(test, 'ABCDEFG')]

In [125]:
blocks

[A	1,0,1~1,2,1,
 B	0,0,2~2,0,2,
 C	0,2,3~2,2,3,
 D	0,0,4~0,2,4,
 E	2,0,5~2,2,5,
 F	0,1,6~2,1,6,
 G	1,1,8~1,1,9]

In [138]:
dropblocks(blocks)

([A	1,0,1~1,2,1,
  B	0,0,2~2,0,2,
  C	0,2,2~2,2,2,
  D	0,0,3~0,2,3,
  E	2,0,3~2,2,3,
  F	0,1,4~2,1,4,
  G	1,1,5~1,1,6],
 9)

In [140]:
def part1(text: list[str]) -> int:
    blocks = [Block.parse(s.strip()) for s in text]
    nblocks, _ = dropblocks(blocks)
    supports = 0
    for i in range(len(nblocks)):
        removed = nblocks[:i] + nblocks[i+1:]
        _, n = dropblocks(removed)
        if n == 0:
            supports += 1
    return supports

In [141]:
part1(test)

5

In [78]:
with open('input', 'rt') as f:
    inp = f.readlines()

In [144]:
with open('output1', 'wt') as f:
    f.write(str(part1(inp)))